In [0]:
# configs = {"fs.azure.account.key.storage001de.blob.core.windows.net": "MA38y0ZA2Ij1L3QXl/F/Y8EGXR1VjTGb896PNbUmPT99uhVLoi0qCEiBEBagBQ4yscqwq9Zxb0vX+AStlr0Z3Q=="}

# # create mount to landing
# dbutils.fs.mount(
# 	source = "wasbs://landing@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/landing",
# 	extra_configs = configs
# )

In [0]:
# # create mount to bronze
# dbutils.fs.mount(
# 	source = "wasbs://bronze@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/bronze",
# 	extra_configs = configs
# )

In [0]:
%fs ls /mnt/datalake/

path,name,size,modificationTime
dbfs:/mnt/datalake/bronze/,bronze/,0,0
dbfs:/mnt/datalake/landing/,landing/,0,0


In [0]:
# Table name
table_movies_metadata = "bronze.movies_metadata_cs"

# bronze location
movies_metadata_landingLocation =  "/mnt/datalake/landing/movies_metadata.csv"
movies_metadata_bronzeLocation = "/mnt/datalake/bronze/movies_metadata"

In [0]:
# Read movie meta data from Data Lake
movies_metadata_DF = spark.read.csv(movies_metadata_landingLocation, header=True, inferSchema=True)
movies_metadata_DF.count()

45572

In [0]:
movies_metadata_DF.printSchema()

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu

In [0]:
 
from delta.tables import *

#check if the bronze location contain the delta table
if (DeltaTable.isDeltaTable(spark, movies_metadata_bronzeLocation)):
    # If Delta table exists, append the data

    # In the bronze zone, we do not do any filtering or remove duplicates
    movies_metadata_DF.write.mode("append").format("delta").save(movies_metadata_bronzeLocation)
else:
    # If no, save the file to bronzeLocation
    movies_metadata_DF.write.mode("overwrite").format("delta").save(movies_metadata_bronzeLocation)
    

In [0]:
# create the schema and table, if required

spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {table_movies_metadata} USING delta LOCATION '{movies_metadata_bronzeLocation}'")

# Note: Using spark.sql because we use f-string to retrieve the processedLocation

DataFrame[]

In [0]:
%sql
DESCRIBE HISTORY bronze.movies_metadata_cs;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2023-11-22T08:52:45Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(845516225391862),1121-120918-uq53wl18,null,WriteSerializable,true,"Map(numFiles -> 4, numOutputRows -> 45572, numOutputBytes -> 17414449)",null,Databricks-Runtime/13.3.x-scala2.12


In [0]:
# Maintenance for Delta table

# To optimized the performance of the Delta table, we need to excute 2 commands:
# 1. optimize(): Optimize the number of files used to store the data
# 2. vacuum(): Remove the old version of the data. This reduces the overhead but it limites the version we can go back to.
movies_metadata_DataDelta = DeltaTable.forName(spark, table_movies_metadata)

if movies_metadata_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	movies_metadata_DataDelta.optimize()
	movies_metadata_DataDelta.vacuum()

In [0]:
%sql
DESCRIBE HISTORY bronze.movies_metadata_cs;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2023-11-22T08:53:08Z,1311347395691033,totnvt02@gmail.com,VACUUM END,Map(status -> COMPLETED),null,List(845516225391862),1121-120918-uq53wl18,1,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/13.3.x-scala2.12
1,2023-11-22T08:53:03Z,1311347395691033,totnvt02@gmail.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(845516225391862),1121-120918-uq53wl18,0,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/13.3.x-scala2.12
0,2023-11-22T08:52:45Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(845516225391862),1121-120918-uq53wl18,null,WriteSerializable,true,"Map(numFiles -> 4, numOutputRows -> 45572, numOutputBytes -> 17414449)",null,Databricks-Runtime/13.3.x-scala2.12


In [0]:
%sql
select count(*) from bronze.movies_metadata_cs

count(1)
45572
